In [1]:
import numpy as np
import tensorflow as tf
from simulator.core.simulator import Simulator
from simulator import Config
from simulator.environment.task_environment import TaskEnvironment
from simulator.logger import LogOutputConsolePrint, Logger
from simulator.world_builds.box_world import BoxWorld
from simulator.reporters import TransitionReporter
from simulator.task_multiplexing import TaskMultiplexerSelectorDql
from simulator.task_multiplexing import TaskMultiplexerSelectorLocal
from simulator.task_multiplexing import TaskMultiplexerSelectorRandom
from simulator.task_multiplexing import TaskMultiplexerSelectorRemote

## Set simulator behaviour

In [2]:
Logger.registerLogOutput(LogOutputConsolePrint())
np.random.seed(Config.get('random_seed'))
tf.random.set_seed(Config.get('random_seed'))

## Creating an instace of Box World

In [3]:

world = BoxWorld()
edgeNodes, mobileNodes = world.build()
edgeReward, mobileReward = world.defaultRewards()


## Creating the reporters

In [4]:
mobileReporter = TransitionReporter() 
edgeReporter = TransitionReporter() 

## Task multiplexing

In [5]:
dqlSelectorGenerator = lambda state: TaskMultiplexerSelectorDql(state, Config.get("dql_training_buffer_size"))
localSelectorGenerator = lambda state: TaskMultiplexerSelectorLocal()
remoteSelectorGenerator = lambda state: TaskMultiplexerSelectorRemote()
randomSelectorGenerator = lambda state: TaskMultiplexerSelectorRandom()

## Prepare and Run the Simulation

In [6]:
taskEnvironment = TaskEnvironment(edgeNodes, mobileNodes, 
                                  edgeSelectorGenerator= dqlSelectorGenerator, 
                                  mobileSelectorGenerator= dqlSelectorGenerator,
                                  edgeRewardFunction= edgeReward,
                                  mobileRewardFunction=mobileReward)
simulator = Simulator()
taskEnvironment.initialize(simulator, [mobileReporter.addTransition], [edgeReporter.addTransition])
simulator.run()

Time: 0, edge load: [1, 0, 0]
Time: 0, edge load: [2, 0, 0]
Time: 0, edge load: [3, 0, 0]
Time: 0, edge load: [4, 0, 0]
Time: 0.6031813352662093, Run Complete: Task//id:1263 size:1000000 workload:596595 nodeId:21 arrivalTime:0.006586335266209328 hopLimit:2 powerConsumed:0.2982975 log:['local run(0.006586335266209328)']
Time: 0.9353961601588627, Run Complete: Task//id:1267 size:1382110 workload:623207 nodeId:39 arrivalTime:0.0740555333888627 hopLimit:2 powerConsumed:0.430670313385 log:['local run(0.0740555333888627)']
Time: 1, edge load: [3, 0, 0]
Time: 1, edge load: [2, 0, 0]
Time: 1, edge load: [4, 0, 0]
Time: 1, edge load: [1, 0, 0]
Time: 1.03603778577206, Run Complete: Task//id:1274 size:1157214 workload:718206 nodeId:79 arrivalTime:0.20491974768805996 hopLimit:2 powerConsumed:0.415559019042 log:['local run(0.20491974768805996)']
Time: 1.105551069936674, Run Complete: Task//id:1268 size:1845109 workload:555399 nodeId:19 arrivalTime:0.08077937644567387 hopLimit:2 powerConsumed:0.5123

## Report delay

In [7]:
print("mobile delay: " + str(mobileReporter.averageDelay()))
print("mobile power consumed: " + str(mobileReporter.averagePowerConsumed()))

mobile delay: 3.356527085322917
mobile power consumed: 0.6727564724217938
